In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils

import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_dir = '../input/aptos2019-blindness-detection/'
test_dir = data_dir + '/test_images/'
label = pd.read_csv(data_dir+"test.csv")
IMG_SIZE = 224

In [ ]:
import sys
path = "../input/efficient-net-deps-1"
sys.path.append(path)
from efficientnet_pytorch import EfficientNet

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
                #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
model = EfficientNet.from_name('efficientnet-b0')
model.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b0-08094119.pth'))
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)
for param in model.parameters():
    param.requires_grad = True
model = model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.00015, weight_decay=1e-5)
loss_func = nn.MSELoss()
train_on_gpu = torch.cuda.is_available()
model.load_state_dict(torch.load("../input/weights/weights.pt"))
model.eval()

In [ ]:
class Imagedata(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = str(self.df[index][0])
        img_path = os.path.join(self.data_dir, img_name+'.png')
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
data_transf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation((-180, 180)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.ColorJitter(brightness=2, contrast=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
test_data = Imagedata(df = label, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, batch_size = 64)

In [ ]:
num = len(pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv"))
test_preds = np.zeros((num, 1))
device = torch.device("cuda:0")
for j in range(5):
    i = 0
    tk0 = tqdm(test_loader)
    for i, x_batch in enumerate(tk0):
        pred = model(x_batch.to(device))
        test_preds[i * 64:(i + 1) * 64] += pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1) * 0.2
test_preds

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)
sample